# Дискриминантный анализ данных

### Задачи

1. **Осуществить кластерный анализ данных в пространстве финансовых коэффициентов и рассчитать вектор классификаций для выборки (выполнено в предыдущей лаборатоной работе).**
2. **С помощью алгоритма классификации на основе линейного дискриминантного анализа классифицировать:**
    - **исходную выборку (режим переклассификации с вычислением условных и безусловных вероятностей ошибок);**
    - **выборку предварительно исключенных наблюдений из различных классов (по 10%) (режим экзамена с вычислением условных и безусловных вероятностей ошибок).**

In [1]:
import sys
import pandas as pd
import numpy as np
import warnings
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

sys.path.append('../src')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from discr_analysis_utils import create_cm_without_test, create_cm_with_test, calculate_errors, deviation_distribution

In [2]:
file_path = '../data/data_clustered.csv'
df = pd.read_csv(file_path)

display(df)

,I,F1,F2,F3,F4,F5,k1,k2,k4,k5,k6,k7,k9,k10,k11,k13,k14,k15,k18,k19,Cluster
0,-0.158751,-0.314829,0.829452,-1.069358,-0.026382,-0.460131,0.000000,0.014084,0.000000,0.797945,0.449936,0.000000,1.000000,0.000000,0.012880,0.062375,0.383869,0.795137,0.002558,0.002752,3
1,0.423813,1.042911,1.147481,-0.177917,0.021606,0.080521,0.245124,0.299176,0.558148,0.910623,0.264507,1.000000,1.000000,0.000000,0.028820,0.082380,1.000000,1.000000,0.003869,0.002987,1
2,-0.333018,-0.612924,1.103026,-0.856950,-0.730159,-1.320378,0.000000,0.000000,0.000000,0.764261,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000528,3
3,0.801346,2.530072,-0.508093,1.008972,0.124333,0.568267,0.809119,0.453227,0.853955,0.940561,0.899900,0.962963,1.000000,0.339213,0.024809,0.714604,1.000000,1.000000,0.005724,0.011632,1
4,-0.167511,0.808507,0.459595,-1.647264,-0.807870,-1.351857,0.194844,0.000000,0.444533,0.864509,0.000000,0.000000,1.000000,0.007876,0.000000,0.000000,0.514314,0.625071,0.002160,0.001286,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,0.334152,0.772825,-0.263806,0.868460,1.244943,-0.826719,0.323204,0.142995,0.397917,0.757061,0.041652,0.874880,0.795362,0.355109,0.051856,0.419410,0.829911,1.000000,0.004824,0.011298,1
2691,0.330281,-0.115035,1.252717,0.646925,0.524918,0.389984,0.143047,0.151816,0.157581,0.830148,0.354400,1.000000,1.000000,0.006451,0.066174,0.380569,1.000000,1.000000,0.001887,0.004243,2
2692,0.173085,-0.099070,0.455273,1.861309,0.615917,-1.803509,0.113393,0.000000,0.190636,0.808592,0.000000,0.000000,0.836614,0.050279,0.045358,0.714604,1.000000,1.000000,0.002285,0.009967,3
2693,0.182451,-0.811812,1.148944,0.794026,0.600394,0.612646,0.077662,0.000000,0.000000,0.748261,0.494889,0.907597,0.868915,0.033910,0.074567,0.439297,0.944861,1.000000,0.001407,0.003355,2


In [3]:
X = df.drop(['Cluster', 'I'] + [f'F{i}' for i in range(1,6)], axis=1)
y = df['Cluster']

## Результаты переклассификации обучающей выборки

In [4]:
lda_reclass = LinearDiscriminantAnalysis()
lda_reclass.fit(X, y) # научили распознавать какой объект какому классу

,solver,'svd'
,shrinkage,None
,priors,None
,n_components,None
,store_covariance,False
,tol,0.0001
,covariance_estimator,None


In [5]:
y_pred_reclass = lda_reclass.predict(X) #предсказанные классы для наблюдений
cm_reclass = confusion_matrix(y, y_pred_reclass)

class_sizes_reclass = cm_reclass.sum(axis=1) #кол-во наблюдений в истинных классах

freq_table_reclass = create_cm_without_test(cm_reclass, class_sizes_reclass)
percent_table_reclass = create_cm_without_test(
    cm_reclass/class_sizes_reclass.reshape(-1,1)*100, 
    (cm_reclass/class_sizes_reclass.reshape(-1,1)*100).sum(axis=1)
)

display(freq_table_reclass)
display(percent_table_reclass)

,Предсказанный 1,Предсказанный 2,Предсказанный 3,Предсказанный 4,Всего наблюдений
Истинный 1,466,36,3,1,506
Истинный 2,7,741,12,50,810
Истинный 3,19,28,670,27,744
Истинный 4,0,8,18,609,635


,Предсказанный 1,Предсказанный 2,Предсказанный 3,Предсказанный 4,Всего наблюдений
Истинный 1,92.1,7.1,0.6,0.2,100.0
Истинный 2,0.9,91.5,1.5,6.2,100.0
Истинный 3,2.6,3.8,90.1,3.6,100.0
Истинный 4,0.0,1.3,2.8,95.9,100.0


### Таблица ошибок классификации


**Согласно информации из таблицы ошибок классификации 92,2% исходных наблюдений классифицированы верно. Наиболее высокая точность достигнута при классификации предприятий, относящихся к четвертому классу платежеспособности (95,9%), наименьшая – предприятий из третьего класса. Это может быть связано, например, с пересекаемостью третьего класса платежеспособности со смежными классами.**

In [6]:
n_samples = len(y) #всего наблюдений
error_table_reclass, unconditional_error_rate_reclass = calculate_errors(class_sizes_reclass, n_samples, cm_reclass)

display(error_table_reclass)
print(f"Безусловная вероятность ошибки P: {unconditional_error_rate_reclass:.3f} ({unconditional_error_rate_reclass*100:.1f}%)")

,Оценка априорной вероятности класса,Оценка условной вероятности ошибки
1,0.188,0.079
2,0.301,0.085
3,0.276,0.099
4,0.236,0.041


Безусловная вероятность ошибки P: 0.078 (7.8%)


### Таблица коэффициентов классифицирующих функций

**Правило классификации следующее: наблюдение принадлежит классу кредитоспособности i , для классифицирующей функции которого di получено наибольшее значение.**

In [7]:
coef_table_reclass = pd.DataFrame(
    lda_reclass.coef_.T,
    index=X.columns,
    columns=[f'd{i}' for i in range(1, 5)]
)

intercept_row_reclass = pd.DataFrame(
    [lda_reclass.intercept_],
    index=['Константа'],
    columns=[f'd{i}' for i in range(1, 5)]
)

lda_coefficients_reclass = pd.concat([coef_table_reclass, intercept_row_reclass])

lda_coefficients_reclass.round(3)

,d1,d2,d3,d4
k1,8.125,-1.868,-2.739,-0.881
k2,4.811,-0.920,-0.856,-1.657
k4,16.307,-4.431,-3.427,-3.328
k5,-3.056,-0.020,2.438,-0.395
k6,-0.416,2.621,-1.671,-1.054
k7,3.372,3.412,-5.237,-0.903
k9,3.097,3.038,5.435,-12.711
k10,-4.353,-0.420,1.067,2.755
k11,3.741,-1.650,1.729,-2.903
k13,1.288,2.011,0.074,-3.678


## Результаты классификации экзаменационной выборки

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)

lda_exam = LinearDiscriminantAnalysis()
lda_exam.fit(X_train, y_train)

y_pred_train_exam = lda_exam.predict(X_train)
y_pred_test_exam = lda_exam.predict(X_test)

cm_train_exam = confusion_matrix(y_train, y_pred_train_exam)
cm_test_exam = confusion_matrix(y_test, y_pred_test_exam)

In [9]:
class_sizes_exam_train = cm_train_exam.sum(axis=1)

test_class_distribution = cm_test_exam.sum(axis=0)
exam_test_sample = list(test_class_distribution) + [test_class_distribution.sum()]

test_total = test_class_distribution.sum()

freq_table_exam_train = create_cm_with_test(cm_train_exam, class_sizes_exam_train, exam_test_sample)
percent_table_exam_train = create_cm_with_test(
    cm_train_exam / class_sizes_exam_train.reshape(-1, 1) * 100, 
    (cm_train_exam / class_sizes_exam_train.reshape(-1, 1) * 100).sum(axis=1), 
    list(test_class_distribution / test_total * 100) + [100.0]
)

display(freq_table_exam_train)
display(percent_table_exam_train)

,Предсказанный 1,Предсказанный 2,Предсказанный 3,Предсказанный 4,Итого
Истинный 1,420,31,3,1,455
Истинный 2,7,665,11,46,729
Истинный 3,19,30,600,21,670
Истинный 4,0,8,16,547,571
Тестовая выборка,45,85,73,67,270


,Предсказанный 1,Предсказанный 2,Предсказанный 3,Предсказанный 4,Итого
Истинный 1,92.3,6.8,0.7,0.2,100.0
Истинный 2,1.0,91.2,1.5,6.3,100.0
Истинный 3,2.8,4.5,89.6,3.1,100.0
Истинный 4,0.0,1.4,2.8,95.8,100.0
Тестовая выборка,16.7,31.5,27.0,24.8,100.0


In [10]:
class_sizes_exam_test = cm_test_exam.sum(axis=1)

freq_table_exam_test = create_cm_without_test(cm_test_exam, class_sizes_exam_test)

percent_table_exam_test = create_cm_without_test(
    cm_test_exam/class_sizes_exam_test.reshape(-1,1)*100, 
    (cm_test_exam/class_sizes_exam_test.reshape(-1,1)*100).sum(axis=1)
)

display(freq_table_exam_test)
display(percent_table_exam_test)

,Предсказанный 1,Предсказанный 2,Предсказанный 3,Предсказанный 4,Всего наблюдений
Истинный 1,45,6,0,0,51
Истинный 2,0,76,1,4,81
Истинный 3,0,3,69,2,74
Истинный 4,0,0,3,61,64


,Предсказанный 1,Предсказанный 2,Предсказанный 3,Предсказанный 4,Всего наблюдений
Истинный 1,88.2,11.8,0.0,0.0,100.0
Истинный 2,0.0,93.8,1.2,4.9,100.0
Истинный 3,0.0,4.1,93.2,2.7,100.0
Истинный 4,0.0,0.0,4.7,95.3,100.0


### Таблица ошибок классификации экзаменационной выборки

**Оценки вероятностей ошибок при классификации экзаменационной выборки в основном немного выше аналогичных оценок для переклассификации исходной выборки. В целом, 93% наблюдений экзаменационной выборки классифицированы верно.**

In [11]:
error_table_exam, unconditional_error_rate_exam = calculate_errors(class_sizes_exam_test, class_sizes_exam_test.sum(), cm_test_exam)

display(error_table_exam)
print(f"Безусловная вероятность ошибки P: {unconditional_error_rate_exam:.3f} ({unconditional_error_rate_exam*100:.1f}%)")

,Оценка априорной вероятности класса,Оценка условной вероятности ошибки
1,0.189,0.118
2,0.300,0.062
3,0.274,0.068
4,0.237,0.047


Безусловная вероятность ошибки P: 0.070 (7.0%)


### Коэффициенты классифицирующих функций

In [12]:
coef_table_exam = pd.DataFrame(
    lda_exam.coef_.T,
    index=X_test.columns,
    columns=[f'd{i}' for i in range(1, 5)]
)

intercept_row_exam = pd.DataFrame(
    [lda_exam.intercept_],
    index=['Константа'],
    columns=[f'd{i}' for i in range(1, 5)]
)

lda_coefficients_exam = pd.concat([coef_table_exam, intercept_row_exam])

display(lda_coefficients_exam.round(3))

,d1,d2,d3,d4
k1,7.921,-1.828,-2.826,-0.661
k2,5.381,-1.085,-0.873,-1.877
k4,15.899,-4.296,-3.325,-3.283
k5,-3.293,-0.098,2.556,-0.250
k6,-0.332,2.573,-1.659,-1.075
k7,3.099,3.395,-5.216,-0.684
k9,3.140,2.988,5.365,-12.612
k10,-4.242,-0.324,0.947,2.683
k11,2.855,-2.241,2.153,-1.941
k13,1.181,1.969,0.142,-3.622


## Отклонения прогнозных рейтингов от «истинных» для различных условий классификации

**В данной таблице систематизирована доля несовпадений прогнозных рейтингов от исходных для различных классов. Процент несовпадений с отклонением на |k|≤1 классов (не более, чем на один класс) при использовании дискриминантного анализ составляет 96,99% и 98,52% для режима переклассификации и режима экзамена соответственно.**

In [13]:
deviation_reclass = deviation_distribution(y, y_pred_reclass)
deviation_exam = deviation_distribution(y_test, y_pred_test_exam)

deviation_summary_table = pd.DataFrame({
    'Режим переклассификации': [deviation_reclass[k] for k in range(-3, 4)] + [deviation_reclass['|k|≤1']],
    'Режим экзамена': [deviation_exam[k] for k in range(-3, 4)] + [deviation_exam['|k|≤1']]
}, index=[-3, -2, -1, 0, 1, 2, 3, '|k|≤1'])

display(deviation_summary_table.T.round(2))

,-3,-2,-1,0,1,2,3,|k|≤1
Режим переклассификации,0.0,1.0,1.97,92.24,2.78,1.97,0.04,96.99
Режим экзамена,0.0,0.0,2.22,92.96,3.33,1.48,0.00,98.52
